In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import torch
# from fake_claims_generator import generate, grab_random_prompts

In [5]:
# !pip3 install git+https://github.com/huggingface/transformers@v3.1.0 # there are errors in the later versions
!pip3 install transformers==3.3.1

     |████████████████████████████████| 1.1MB 17.4MB/s 
     |████████████████████████████████| 890kB 49.9MB/s 
     |████████████████████████████████| 1.2MB 54.4MB/s 
     |████████████████████████████████| 3.0MB 48.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=93c2688af1cc8bb03b418d1d9513b3ba9e4423ba43c91f34a2e6fd4263735e46
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [7]:
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [8]:
def grab_random_prompts(data, numprompts, first_words=5, column='claims'):
    '''
    Inputs:
    numprompts int: Number of prompts we would want
    first_words int: How many first words would we want
    '''
    num_data = len(data)
    prompts = []
    while len(prompts) < numprompts:
        row = np.random.randint(0, num_data-1)
        try:
            words = data[column][row].split()
            firsts = " ".join(words[:first_words])
            prompts.append(firsts)
        except:
            #print("No claims")
            pass
    
#     for indx in rows:
#         words = data[column][indx].split()
#         firsts = " ".join(words[:first_words])
#         prompts.append(firsts)
    return prompts
# *** Modify the directory to the right model ***
OUTPUT_DIR = "./Models/Experiments/Claims/3epochs"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)
def generate(input_str, length=250, n=5):
    cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
    model.eval()
    with torch.no_grad():
        for i in range(length):
            outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0)
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        return output_text

def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [9]:
device

'cuda'

## Load Real Data and Label Them


In [10]:
data = pd.read_csv("./Models/Data/H01L.csv")
real_data = pd.DataFrame({"Claims": data["claims"]})
real_data["Label"] = 0
real_data = real_data[real_data['Claims'].notnull()]
# real_data = pd.DataFrame({"Abstracts": data["abstracts"]})
# real_data["Label"] = 0
# real_data = real_data[real_data['Abstracts'].notnull()]

In [11]:
real_data

,Claims,Label
0,1. An organic light emitting diode display co...,0
1,1. An electric element cooling module compris...,0
2,2. The organic light emitting diode display o...,0
3,2. The electric element cooling module of cla...,0
4,3. The organic light emitting diode display o...,0
...,...,...
8941,"2. The method of claim 1, further comprising:...",0
8942,"3. The method of claim 1, wherein the first g...",0
8943,"4. The method of claim 1, wherein a width of ...",0
8944,"5. The method of claim 1, wherein a length of...",0


## Generate Fake Data and Label Them

In [ ]:
# How many fake claims do you want to generate
# num_fake_claims = len(real_data)
# prompts = grab_random_prompts(data, num_fake_claims, first_words=5)
# fake_claims = []
# for i, prompt in enumerate(prompts):
#     print(f"Generating Prompt {i+1} ... Prompt: {prompt}")
#     generated_text = generate(prompt, n=8)
#     fake_claims.append(generated_text)
  

In [ ]:
# write in chunks, and write to csv
'''
This code block is responsible for generating fake patent Claims
I first deine the file paths where we will save the fake patent data
Then, I generate the fake patent prompts by sampling from the real abstracts
I save the prompts into a csv file.
Since I am running the generation on Colab and I'm generating thousands of fake patent Claims,
there is a high frequency of interuption, I therefore need to save the prompts (to be accessed later),
and save the patent claims for every 100 that are generated.
The last_interupted variable stores where the generation was last interupted.
'''
import os.path
fake_data_file = "Models/Experiments/Claims/fake_patent_claims_3epochs.csv"
prompt_file = "Models/Experiments/Claims/prompts_3epochs.csv"
# fake_data_file = "Models/Experiments/Abstracts/fake_patent_abstracts_3epochs.csv"
# prompt_file = "Models/Experiments/Abstracts/prompts_3epochs.csv"
num_fake_claims = len(real_data)
if os.path.exists(prompt_file):
  prompts = pd.read_csv(prompt_file)
  prompts = prompts['Prompts'].tolist()
else:
  prompts = grab_random_prompts(data, num_fake_claims, first_words=5)
  # save prompts
  prompt_df = pd.DataFrame({"Prompts": prompts})
  prompt_df.to_csv(prompt_file)
fake_claims = []
# modify last interrupted 
last_interrupted = 8200
for i, prompt in enumerate(prompts[last_interrupted:]): # modify last_interrupted if we got disconnected
    if i % 100 == 0 and i != 0: # for every 100 claims
      df = pd.DataFrame({"Claims": fake_claims, "Label": 1})
      if os.path.exists(fake_data_file): # this means this csv is already created
        df.to_csv(fake_data_file, mode='a', header=False)
      else:
        df.to_csv(fake_data_file)
      fake_claims = []
    print(f"Generating Prompt {i+1} ... Prompt: {prompt}")
    generated_text = generate(prompt, n=8)
    fake_claims.append(generated_text)
# last batch
df = pd.DataFrame({"Claims": fake_claims, "Label": 1})
df.to_csv(fake_data_file, mode='a', header=False)

Generating Prompt 1 ... Prompt: 1. A method for forming
Generating Prompt 2 ... Prompt: 6. The memory of claim
Generating Prompt 3 ... Prompt: 20. The organic light emitting
Generating Prompt 4 ... Prompt: 9. The method of claim
Generating Prompt 5 ... Prompt: 7. A piezoelectrically actuated valve
Generating Prompt 6 ... Prompt: 40. The method of claim
Generating Prompt 7 ... Prompt: 1. An article comprising: a
Generating Prompt 8 ... Prompt: 4. The method according to
Generating Prompt 9 ... Prompt: 16. The apparatus of claim
Generating Prompt 10 ... Prompt: 16. A vibration-type actuator comprising:
Generating Prompt 11 ... Prompt: 21. The method of manufacturing
Generating Prompt 12 ... Prompt: 13. The semiconductor device according
Generating Prompt 13 ... Prompt: 12. The method of claim
Generating Prompt 14 ... Prompt: 13. The system of claim
Generating Prompt 15 ... Prompt: 14. A sensor package comprising:
Generating Prompt 16 ... Prompt: 11. An encapsulant formed by
Generating Pr

In [12]:
fake_data = pd.read_csv("./Models/Experiments/Claims/fake_patent_claims_3epochs.csv")
# fake_data = pd.read_csv("./Models/Experiments/Abstracts/fake_patent_abstracts_3epochs.csv")
fake_data = fake_data.loc[:, ~fake_data.columns.str.contains('^Unnamed')]
fake_data.head()

,Claims,Label
0,19. An emitter comprising: a first emitter com...,1
1,"14. The method of claim 6, wherein the second ...",1
2,"5. The package according to claim 2, wherein t...",1
3,7. The organic light emitting display componen...,1
4,10. The method for manufacturing a plurality o...,1


In [13]:
len(fake_data)

8420

In [14]:
frames = [real_data, fake_data]
full_data = pd.concat(frames)


full_data = full_data.sample(frac=1).reset_index(drop=True)


In [17]:
full_data


,Claims,Label
0,"10. The method of claim 11, wherein the first ...",1
1,18. The flow controller according to claim 17...,0
2,"17. The method according to claim 10, wherein...",0
3,"13. The solar cell of claim 12, further compri...",1
4,"5. The display device of claim 1, wherein the ...",1
...,...,...
16835,"2. The method of claim 1, further comprising ...",0
16836,"3. The device according to claim 2, further c...",0
16837,10. The apparatus of claim 10 wherein the seco...,1
16838,"15. A pixel assembly according to claim 19, wh...",1


In [21]:
full_data['Claims'][0][:250]

'10. The method of claim 11, wherein the first electrode comprises a metal layer and the second electrode comprises a metal material. <EOS>\n<BOS> nan <EOS>\n<BOS> 9.  The organic light emitting diode as claimed in claim 4, further comprising: a first e'

In [16]:
# path: "Models/Experiments/Claims"
full_data.to_csv("./Models/Experiments/Claims/3epochs/real_fake_claims.csv")